In [438]:
import json

import numpy as np

tank_param_bounds: dict = {
    # tank-0
    "t0_is": {"min": 0.01, "max": 100},
    "t0_boc": {"min": 0.1, "max": 0.5},
    "t0_soc_uo": {"min": 0.1, "max": 0.5},
    "t0_soc_lo": {"min": 0.1, "max": 0.5},
    "t0_soh_uo": {"min": 75, "max": 100},
    "t0_soh_lo": {"min": 0, "max": 50},

    # tank-1
    "t1_is": {"min": 0.01, "max": 100},
    "t1_boc": {"min": 0.01, "max": 0.5},
    "t1_soc": {"min": 0.01, "max": 0.5},
    "t1_soh": {"min": 0, "max": 100},

    # tank-2
    "t2_is": {"min": 0.01, "max": 100},
    "t2_boc": {"min": 0.01, "max": 0.5},
    "t2_soc": {"min": 0.01, "max": 0.5},
    "t2_soh": {"min": 0, "max": 100},

    # tank-3
    "t3_is": {"min": 0.01, "max": 100},
    "t3_soc": {"min": 0.01, "max": 0.5}
}
tank_lb: np.ndarray = np.array([
    # [Tank-0] 
    tank_param_bounds['t0_is']['min'],
    tank_param_bounds['t0_boc']['min'],
    tank_param_bounds['t0_soc_uo']['min'],
    tank_param_bounds['t0_soc_lo']['min'],
    tank_param_bounds['t0_soh_uo']['min'],
    tank_param_bounds['t0_soh_lo']['min'],

    # [Tank-1]
    tank_param_bounds['t1_is']['min'],
    tank_param_bounds['t1_boc']['min'],
    tank_param_bounds['t1_soc']['min'],
    tank_param_bounds['t1_soh']['min'],

    # [Tank-2]
    tank_param_bounds['t2_is']['min'],
    tank_param_bounds['t2_boc']['min'],
    tank_param_bounds['t2_soc']['min'],
    tank_param_bounds['t2_soh']['min'],

    # [Tank-3]
    tank_param_bounds['t3_is']['min'],
    tank_param_bounds['t3_soc']['min'],

])
muskingum_param_bound: dict = {
    "k": {"min": 0, "max": 5},
    "x": {"min": 0, "max": 0.5}

}
muskingum_lb: np.ndarray = np.array([
    muskingum_param_bound["k"]["min"],
    muskingum_param_bound["x"]["min"],
])
# muskingum parameter upper bounds
muskingum_ub: np.ndarray = np.array([
    muskingum_param_bound["k"]["max"],
    muskingum_param_bound["x"]["max"],
])

basin_default = tank_lb
channel_default = 0.5 * (muskingum_lb + muskingum_ub)
parsed_node = dict()
print(basin_default)

[1.0e-02 1.0e-01 1.0e-01 1.0e-01 7.5e+01 0.0e+00 1.0e-02 1.0e-02 1.0e-02
 0.0e+00 1.0e-02 1.0e-02 1.0e-02 0.0e+00 1.0e-02 1.0e-02]


In [439]:
basin = open('CedarCreek.basin', 'r').read()
print(basin)

Basin: CedarCreek
     Description: HMS Model For Cedar Creek
     Last Modified Date: 1 November 2010
     Last Modified Time: 18:18:23
     Version: 4.10
     Filepath Separator: \
     Unit System: English
     Grid Cell File: CedarCreek.mod
     Missing Flow To Zero: No
     Enable Flow Ratio: No
     Compute Local Flow At Junctions: No
     Unregulated Output Required: No

     Enable Sediment Routing: No

     Enable Quality Routing: No
End:

Subbasin: W310
     Last Modified Date: 19 February 2023
     Last Modified Time: 02:40:17
     Canvas X: 658813.695603162
     Canvas Y: 4566675.75032761
     Label X: 16.0
     Label Y: 16.0
     Area: 6.3298
     Downstream: Outlet1

     Discretization: None
     File: CedarCreek.sqlite

     Canopy: None
     Allow Simultaneous Precip Et: No
     Plant Uptake Method: None

     Surface: None

     LossRate: SCS
     Percent Impervious Area: 0.0
     Curve Number: 78.870
     Initial Abstraction: 0

     Transform: SCS
     Lag: 143.19
 

In [440]:
def extract_data(file_content):
    sections = file_content.split("End:\n")
    data = []

    for section in sections:
        lines = section.strip().split("\n")
        section_data = {}
        current_key = None

        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                section_data[key.strip()] = value.strip()
                current_key = key.strip()
            elif line.strip() and current_key:
                section_data[current_key] += " " + line.strip()

        if section_data:
            data.append(section_data)

    return data

In [441]:
data = extract_data(basin)
for section in data:
    print(section)

{'Basin': 'CedarCreek', 'Description': 'HMS Model For Cedar Creek', 'Last Modified Date': '1 November 2010', 'Last Modified Time': '18:18:23', 'Version': '4.10', 'Filepath Separator': '\\', 'Unit System': 'English', 'Grid Cell File': 'CedarCreek.mod', 'Missing Flow To Zero': 'No', 'Enable Flow Ratio': 'No', 'Compute Local Flow At Junctions': 'No', 'Unregulated Output Required': 'No', 'Enable Sediment Routing': 'No', 'Enable Quality Routing': 'No'}
{'Subbasin': 'W310', 'Last Modified Date': '19 February 2023', 'Last Modified Time': '02:40:17', 'Canvas X': '658813.695603162', 'Canvas Y': '4566675.75032761', 'Label X': '16.0', 'Label Y': '16.0', 'Area': '6.3298', 'Downstream': 'Outlet1', 'Discretization': 'None', 'File': 'CedarCreek.sqlite', 'Canopy': 'None', 'Allow Simultaneous Precip Et': 'No', 'Plant Uptake Method': 'None', 'Surface': 'None', 'LossRate': 'SCS', 'Percent Impervious Area': '0.0', 'Curve Number': '78.870', 'Initial Abstraction': '0', 'Transform': 'SCS', 'Lag': '143.19', '

In [442]:
basin = {
    'basin_def': {

    },
    'root_node': [

    ]
}
print(basin)

{'basin_def': {}, 'root_node': []}


In [443]:
TANK_PARAMETER_ORDER: list = [
    # T-0fa
    't0_is', 't0_boc', 't0_soc_uo', 't0_soc_lo', 't0_soh_uo', 't0_soh_lo',
    # T-1
    't1_is', 't1_boc', 't1_soc', 't1_soh',
    # T-2
    't2_is', 't2_boc', 't2_soc', 't2_soh',
    # T-3
    't3_is', 't3_soc'
]


def muskingum_param_list2dict(parameters: list) -> dict:
    return {
        "k": parameters[0],
        "x": parameters[1],
    }


def tank_param_list2dict(parameters: list) -> dict:
    # converts flattened parameter to a dict based on TANK_PARAMETER_ORDER

    return {
        parameter_name: parameters[i]
        for i, parameter_name in enumerate(TANK_PARAMETER_ORDER)
    }

In [444]:
sel_nodes = ["Subbasin", "Reach", "Junction", "Sink"]
generel_props = ["Downstream", "Area", "Computation Point"]
numeric_props = ["Area"]
basin_def = {}
for section in data:

    name = None
    component = {}

    for k, v in section.items():
        if k == 'Subbasin':
            name = v
            component['type'] = 'Subbasin'
            component['parameters'] = tank_param_list2dict(list(basin_default))
        if k == 'Reach':
            name = v
            component['type'] = 'Reach'
            component['parameters'] = muskingum_param_list2dict(list(channel_default))
        if k == 'Junction' or k == 'Sink':
            name = v
            component['type'] = k
            component['parameters'] = {}

        if k == 'Area':
            component[k.lower()] = float(v)

        if k == 'Upstream':
            component[k.lower()] = v

        if k == 'Downstream':
            component[k.lower()] = v

    if name is not None:
        basin_def[name] = component
    # for s in section:
    #     if s in sel_nodes:
    #         component = {'type': s}
    # 
    #         if s == 'Subbasin':
    #             component['parameters'] = tank_param_list2dict(list(basin_default))
    # 
    #         if s == 'Reach':
    #             component['parameters'] = muskingum_param_list2dict(list(channel_default))
    #             
    #         basin_def[section[s]] = component

basin['basin_def'] = basin_def

for node in basin['basin_def']:
    ds = basin['basin_def'][node].get('downstream', None)

    if ds is None:
        #  this is root node || needs to be changed
        # no basin will have multiple root node
        if basin.get('root_node', None) is None:
            basin['root_node'] = [node]
        else:
            basin['root_node'].append(node)

    else:
        if basin['basin_def'][ds].get('upstream', None) is None:

            basin['basin_def'][ds]['upstream'] = [node]
        else:
            basin['basin_def'][ds]['upstream'].append(node)

import json

with open('basin.json', "w", encoding="utf-8") as json_file:
    json.dump(basin, json_file, ensure_ascii=False, indent=4)


In [445]:
from queue import Queue


def build_computation_stack(project: dict) -> list:
    computation_stack = []

    node_queue: Queue[str] = Queue()

    # en-queue root note
    for root_node in project['root_node']:
        node_queue.put(root_node)

    while not node_queue.empty():

        # deque node
        node = node_queue.get()

        # add node to the top of computation stack
        computation_stack.append(node)

        # get child nodes and en-queue child nodes
        if project['basin_def'][node].get('upstream', False):
            child_nodes = project['basin_def'][node]['upstream']

            for child in child_nodes:
                node_queue.put(child)

    return computation_stack


print(build_computation_stack(basin))

['Outlet1', 'W310', 'R160', 'J36', 'W300', 'W290', 'R140', 'J44', 'W260', 'R120', 'R130', 'J41', 'J47', 'W270', 'W240', 'W250', 'R80', 'R90', 'J52', 'J57', 'W230', 'W220', 'W200', 'W190', 'R40', 'J62', 'W180', 'W170']


In [446]:
computation_stack = build_computation_stack(basin)
print(computation_stack)

['Outlet1', 'W310', 'R160', 'J36', 'W300', 'W290', 'R140', 'J44', 'W260', 'R120', 'R130', 'J41', 'J47', 'W270', 'W240', 'W250', 'R80', 'R90', 'J52', 'J57', 'W230', 'W220', 'W200', 'W190', 'R40', 'J62', 'W180', 'W170']


In [447]:
import pandas as pd

df = pd.read_csv('data_example.csv')
df['Date'] = pd.to_datetime(df['Date'], utc=True)
df

,Date,Temp,Q,P,E
0,2016-10-09 00:00:00+00:00,15.4,0.25694,0.0,2.79
1,2016-10-10 00:00:00+00:00,14.4,0.25812,0.0,3.46
2,2016-10-11 00:00:00+00:00,14.9,0.30983,0.0,3.65
3,2016-10-12 00:00:00+00:00,16.1,0.31422,0.0,3.46
4,2016-10-13 00:00:00+00:00,20.1,0.30866,0.0,5.64
...,...,...,...,...,...
717,2018-09-26 00:00:00+00:00,17.3,0.34353,0.0,2.17
718,2018-09-27 00:00:00+00:00,13.8,0.33240,0.0,3.68
719,2018-09-28 00:00:00+00:00,15.1,0.36222,0.0,3.79
720,2018-09-29 00:00:00+00:00,17.5,0.34574,0.0,4.02


In [448]:
proc_df = df.copy()
proc_df

,Date,Temp,Q,P,E
0,2016-10-09 00:00:00+00:00,15.4,0.25694,0.0,2.79
1,2016-10-10 00:00:00+00:00,14.4,0.25812,0.0,3.46
2,2016-10-11 00:00:00+00:00,14.9,0.30983,0.0,3.65
3,2016-10-12 00:00:00+00:00,16.1,0.31422,0.0,3.46
4,2016-10-13 00:00:00+00:00,20.1,0.30866,0.0,5.64
...,...,...,...,...,...
717,2018-09-26 00:00:00+00:00,17.3,0.34353,0.0,2.17
718,2018-09-27 00:00:00+00:00,13.8,0.33240,0.0,3.68
719,2018-09-28 00:00:00+00:00,15.1,0.36222,0.0,3.79
720,2018-09-29 00:00:00+00:00,17.5,0.34574,0.0,4.02


In [449]:
def tank_discharge(
        precipitation: np.ndarray,
        evapotranspiration: np.ndarray,
        interval: float,
        area: float,
        t0_is: float, t0_boc: float,
        t0_soc_uo: float, t0_soc_lo: float,
        t0_soh_uo: float, t0_soh_lo: float,
        t1_is: float, t1_boc: float, t1_soc: float, t1_soh: float,
        t2_is: float, t2_boc: float, t2_soc: float, t2_soh: float,
        t3_is: float, t3_soc: float
):
    # TODO kiểm tra số lượng record giữa P và E có bằng nhau không

    if t0_soh_uo < t0_soh_lo:
        print('WARNING-TANK-01: Invalid parameter upper outlet height is less than lower outlet height (Tank 0)')

    time_step = precipitation.shape[0]
    tank_storage = np.zeros((time_step, 4), dtype=np.float64)
    side_outlet_flow = np.zeros((time_step, 4), dtype=np.float64)
    bottom_outlet_flow = np.zeros((time_step, 3), dtype=np.float64)

    # Difference of precipitation & evapotranspiration [only inflow to Tank 0]
    del_rf_et = precipitation - evapotranspiration

    # set initial tank storages | set to 0 if negative value
    tank_storage[0, 0] = max(t0_is, 0)
    tank_storage[0, 1] = max(t1_is, 0)
    tank_storage[0, 2] = max(t2_is, 0)
    tank_storage[0, 3] = max(t3_is, 0)

    for t in np.arange(time_step):
        # TANK 0 : surface runoff
        side_outlet_flow[t, 0] = t0_soc_lo * max(tank_storage[t, 0] - t0_soh_lo, 0) \
                                 + t0_soc_uo * max(tank_storage[t, 0] - t0_soh_uo, 0)

        # TANK 1 : intermediate runoff
        side_outlet_flow[t, 1] = t1_soc * max(tank_storage[t, 1] - t1_soh, 0)

        # TANK 2 : sub-base runoff
        side_outlet_flow[t, 2] = t2_soc * max(tank_storage[t, 2] - t2_soh, 0)

        # TANK 3 : base-flow | Side outlet height = 0
        side_outlet_flow[t, 3] = t3_soc * tank_storage[t, 3]

        bottom_outlet_flow[t, 0] = t0_boc * tank_storage[t, 0]
        bottom_outlet_flow[t, 1] = t1_boc * tank_storage[t, 1]
        bottom_outlet_flow[t, 2] = t2_boc * tank_storage[t, 2]

        if t < (time_step - 1):
            tank_storage[t + 1, 0] = tank_storage[t, 0] + del_rf_et[t + 1] - (
                    side_outlet_flow[t, 0] + bottom_outlet_flow[t, 0])

            tank_storage[t + 1, 1] = tank_storage[t, 1] + bottom_outlet_flow[t, 0] - (
                    side_outlet_flow[t, 1] + bottom_outlet_flow[t, 1])

            tank_storage[t + 1, 2] = tank_storage[t, 2] + bottom_outlet_flow[t, 1] - (
                    side_outlet_flow[t, 2] + bottom_outlet_flow[t, 2])

            tank_storage[t + 1, 3] = tank_storage[t, 3] + bottom_outlet_flow[t, 2] - side_outlet_flow[t, 3]

            # Set tank storage = 0 if tank storage is negative

            tank_storage[t + 1, 0] = max(tank_storage[t + 1, 0], 0)
            tank_storage[t + 1, 1] = max(tank_storage[t + 1, 1], 0)
            tank_storage[t + 1, 2] = max(tank_storage[t + 1, 2], 0)
            tank_storage[t + 1, 3] = max(tank_storage[t + 1, 3], 0)

        for i in range(4):

            total_tank_outflow = bottom_outlet_flow[t, i] + side_outlet_flow[t, i] if i <= 2 else side_outlet_flow[t, i]

            if total_tank_outflow > tank_storage[t, i]:
                print(f'WARNING-TANK-02: Total outlet flow exceeded tank storage for tank {i} at timestep {t}')

    UNIT_CONV_COEFF = (area * 1000) / (interval * 3600)

    discharge = UNIT_CONV_COEFF * side_outlet_flow.sum(axis=1)

    states = dict(
        tank_storage=tank_storage,
        side_outlet_flow=side_outlet_flow,
        bottom_outlet_flow=bottom_outlet_flow
    )

    return discharge, states


def muskingum(in_flow: np.ndarray, del_t: float, k: float, x: float) -> np.ndarray:
    # calculate time-step
    n_step: int = in_flow.shape[0]

    # create a zero array of out_flow
    out_flow: np.ndarray = np.zeros(n_step, dtype=np.float64)

    C0: float = (-k * x + 0.5 * del_t) / (k * (1 - x) + 0.5 * del_t)
    C1: float = (k * x + 0.5 * del_t) / (k * (1 - x) + 0.5 * del_t)
    C2: float = (k * (1 - x) - 0.5 * del_t) / (k * (1 - x) + 0.5 * del_t)

    # constraints check
    if (C0 + C1 + C2) > 1 or x > 0.5 or (del_t / k + x) > 1:
        print("WARNING-MUSKINGUM-01: violates k, x constraints")

    # initial condition
    out_flow[0] = in_flow[0]

    for t in np.arange(1, n_step):
        out_flow[t] = C0 * in_flow[t] + C1 * in_flow[t - 1] + C2 * out_flow[t - 1]

    return out_flow


In [450]:
n_step = len(proc_df['P'].to_numpy())

computation_result = pd.DataFrame(index=proc_df['P'].index)
model_states = dict(
    time=proc_df['P'].index.to_numpy()
)

print(computation_stack)
while len(computation_stack) > 0:
    current_node = computation_stack.pop()
    current_node_def = basin['basin_def'][current_node]

    if current_node_def['type'] == 'Subbasin':
        computation_result[current_node], basin_states = tank_discharge(
            proc_df['P'].to_numpy(),
            proc_df['E'].to_numpy(),
            24.0,
            current_node_def['area'],
            **current_node_def['parameters']
        )
        model_states[current_node] = basin_states


    elif current_node_def['type'] == 'Reach':
        sum_node = np.zeros(n_step, dtype=np.float64)
        for us_node_name in current_node_def['upstream']:
            sum_node += muskingum(
                in_flow=computation_result[us_node_name].to_numpy(),
                del_t=24.0,
                **current_node_def['parameters']
            )
        computation_result[current_node] = sum_node

    elif current_node_def['type'] in ['Sink', 'Junction']:

        sum_node = np.zeros(n_step, dtype=np.float64)

        for us_node_name in current_node_def['upstream']:
            sum_node += computation_result[us_node_name].to_numpy()

        computation_result[current_node] = sum_node

# computation_result.to_csv('computation_result.csv')
for k, v in model_states.items():
    if k == 'time':
        continue

    print(k, v['side_outlet_flow'])
    pd.DataFrame(v['side_outlet_flow']).to_csv(f'{k}.out.csv')
    
        

['Outlet1', 'W310', 'R160', 'J36', 'W300', 'W290', 'R140', 'J44', 'W260', 'R120', 'R130', 'J41', 'J47', 'W270', 'W240', 'W250', 'R80', 'R90', 'J52', 'J57', 'W230', 'W220', 'W200', 'W190', 'R40', 'J62', 'W180', 'W170']
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
WARNING-MUSKINGUM-01: violates k, x constraints
W170 [[1.00000000e-03 1.00000000e-04 1.00000000e-04 1.00000000e-04]
 [0.00000000e+00 1.08000000e-04 9.90000000e-05 1.00000000e-04]
 [0.00000000e+00 1.05840000e-04 9.81000000e-05 9.99900000e-05]
 ...
 [0.00000000e+00 1.71206851e-02 2.44802776e-02 4.32396395e-02]
 [0.00000000e+00 1.67782714e-02 2.41618789e-02 4.30520459e-02]
 [0.00000000e+00 1.64427060e-02 2.38464241e-02 4.28631442e-02]]
W180 [[1.00000000e-03 1.00000000e-04 1.00000000e-04 1.000